In [1]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import train_test_split

# 데이터 불러오기

In [2]:
df = pd.read_csv('분석용데이터_utf_spi수정완료.csv')

In [3]:
#Date를 index로 넣어줌
df.index = df['Date']
df.drop(['Date'], axis = 1, inplace = True)

In [4]:
df

,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,효준감성점수,spi
Date,,,,,,,,
201101월,1676.4,4.52,2.75,3.4,2069.73,1119.25,1856.746125,81.3
201102월,1674.4,4.72,2.75,3.9,1939.30,1124.65,2046.702906,82.5
201103월,1677.5,4.54,3.00,4.1,2106.70,1095.50,1715.795289,83.4
201104월,1684.8,4.54,3.00,3.8,2192.36,1068.00,1635.262791,83.9
201105월,1690.5,4.44,3.00,3.9,2142.47,1078.00,1922.518709,84.1
...,...,...,...,...,...,...,...,...
202008월,3100.4,2.19,0.50,0.7,2326.17,1187.94,1221.939746,113.1
202009월,3115.2,2.24,0.50,1.0,2327.89,1164.65,1208.597571,114.0
202010월,3152.8,2.24,0.50,0.1,2267.15,1136.49,1001.099665,115.6


# 전처리

In [5]:
df['next_spi'] = df['spi'].shift(-1)
df['next_spi'][-1] = 124.9

In [6]:
#전처리에 사용되었던 더미 변수들 drop
df.drop(['spi'], axis = 1, inplace = True)

In [7]:
X = df.copy()
y = X.pop('next_spi')

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [10]:
X_train_new = X_train.drop(['효준감성점수'], axis = 1)
X_test_new = X_test.drop(['효준감성점수'], axis = 1)

# 효준이 제외 기본 성능

In [23]:
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint

In [12]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)

In [13]:
regressor.fit(X_train, y_train)  

RandomForestRegressor(random_state=0)

In [25]:
print('Parameters currently in use:\n')
pprint(regressor.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}


In [14]:
y_pred = regressor.predict(X_test)  # test the output by changing values

In [15]:
from sklearn.metrics import mean_squared_error


In [16]:
RMSE = mean_squared_error(y_test, y_pred)**0.5

In [17]:
RMSE

1.517531043065262

In [26]:
from sklearn.model_selection import RandomizedSearchCV

In [36]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(3, 20, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [1,2, 5, 8,10, 15]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]
# Method of selecting samples for training each tree
# bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

pprint(random_grid)


{'max_depth': [3, 6, 9, 13, 16, 20],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 5, 10],
 'min_samples_split': [1, 2, 5, 8, 10, 15],
 'n_estimators': [100,
                  200,
                  300,
                  400,
                  500,
                  600,
                  700,
                  800,
                  900,
                  1000,
                  1100,
                  1200]}


In [37]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = 1)

# Fit the random search model
# In order to test these models I will need to do a train test split with the training data-set. 
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END max_depth=13, max_features=auto, min_samples_leaf=1, min_samples_split=8, n_estimators=1200; total time=   0.5s
[CV] END max_depth=13, max_features=auto, min_samples_leaf=1, min_samples_split=8, n_estimators=1200; total time=   0.5s
[CV] END max_depth=13, max_features=auto, min_samples_leaf=1, min_samples_split=8, n_estimators=1200; total time=   0.5s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=   0.2s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=   0.2s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=   0.2s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=10, n_estimators=300; total time=   0.1s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=10, n_estimator

/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/joblib/parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/jinwooahn/minif

[CV] END max_depth=16, max_features=auto, min_samples_leaf=2, min_samples_split=15, n_estimators=600; total time=   0.2s
[CV] END max_depth=16, max_features=auto, min_samples_leaf=2, min_samples_split=15, n_estimators=600; total time=   0.2s
[CV] END max_depth=16, max_features=auto, min_samples_leaf=2, min_samples_split=15, n_estimators=600; total time=   0.2s
[CV] END max_depth=13, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=300; total time=   0.1s
[CV] END max_depth=13, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=300; total time=   0.1s
[CV] END max_depth=13, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=300; total time=   0.1s
[CV] END max_depth=16, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   0.5s
[CV] END max_depth=16, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   0.5s
[CV] END max_depth=16, max_featu

/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/joblib/parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/jinwooahn/minif

[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=700; total time=   0.3s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=700; total time=   0.3s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=700; total time=   0.3s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=15, n_estimators=1100; total time=   0.4s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=15, n_estimators=1100; total time=   0.4s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=15, n_estimators=1100; total time=   0.4s
[CV] END max_depth=13, max_features=auto, min_samples_leaf=2, min_samples_split=1, n_estimators=800; total time=   0.1s
[CV] END max_depth=13, max_features=auto, min_samples_leaf=2, min_samples_split=1, n_estimators=800; total time=   0.1s


...

[CV] END max_depth=13, max_features=sqrt, min_samples_leaf=10, min_samples_split=15, n_estimators=800; total time=   0.3s
[CV] END max_depth=13, max_features=sqrt, min_samples_leaf=10, min_samples_split=15, n_estimators=800; total time=   0.3s
[CV] END max_depth=13, max_features=sqrt, min_samples_leaf=10, min_samples_split=15, n_estimators=800; total time=   0.3s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=10, min_samples_split=15, n_estimators=400; total time=   0.2s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=10, min_samples_split=15, n_estimators=400; total time=   0.2s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=10, min_samples_split=15, n_estimators=400; total time=   0.2s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=10, min_samples_split=15, n_estimators=1100; total time=   0.4s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=10, min_samples_split=15, n_estimators=1100; total time=   0.4s
[CV] END max_depth=20, ma

/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/joblib/parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/jinwooahn/minif

[CV] END max_depth=9, max_features=auto, min_samples_leaf=5, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=9, max_features=auto, min_samples_leaf=5, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=9, max_features=auto, min_samples_leaf=5, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=5, n_estimators=300; total time=   0.1s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=5, n_estimators=300; total time=   0.1s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=5, min_samples_split=5, n_estimators=300; total time=   0.1s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=600; total time=   0.2s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=600; total time=   0.2s
[CV] END max_depth=3, max_features=auto, min_sam

/Users/jinwooahn/miniforge3/envs/dev_2/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.97520872 0.97029108 0.9380872  0.8844512         nan 0.96791259
 0.93086716 0.97656792 0.92015097 0.91333484 0.96789004        nan
 0.93735715 0.96716442        nan 0.97665949 0.88511199 0.81328764
 0.91704918 0.97355105 0.967621   0.88511199 0.93906961 0.94491452
 0.93755543 0.97182618 0.93609269 0.96728851        nan 0.93857264
 0.96794091 0.81197073 0.81356893 0.84460499 0.94178866 0.94410529
 0.94495953        nan 0.91015423        nan 0.94172126 0.92924828
 0.93875071 0.95342593 0.93827361 0.94215796        nan 0.81356893
 0.81321533 0.84659615 0.81328764 0.93643908 0.98014229 0.84466995
 0.86461061 0.84460499        nan 0.81302004        nan 0.9742569
 0.93568463 0.84460499 0.93782716 0.84659615 0.93827361 0.95997193
 0.92575096 0.93837429 0.88461854 0.84659615 0.81321684 0.93158543
 0.9387309  0.97659713 0.86473

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=0),
                   n_iter=100, n_jobs=1,
                   param_distributions={'max_depth': [3, 6, 9, 13, 16, 20],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [1, 2, 5, 8, 10,
                                                              15],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100,
                                                         1200]},
                   random_state=42, verbose=2)

In [30]:
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 30}

In [34]:
# Evaluation of Random Search
def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = np.sqrt(mean_squared_error(y_test, predictions))
    print('Model Performance')
    print('RMSE of: ', errors)
    
    return errors

In [35]:
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

best_random = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
best_random.fit(X_train , y_train)

random_accuracy = evaluate(best_random, X_test, y_test)

print('\n')
print('Base Accuracy: ', base_accuracy)
print('\n')
print('Random Accuracy: ', random_accuracy)
print('Improvement of {:0.2f}%.'.format((random_accuracy - base_accuracy) / base_accuracy))

print('\n')
print('RF_Randomized_Search_CV')
print('\n')

Model Performance
RMSE of:  1.6532775528224812
Model Performance
RMSE of:  1.8348417320461654


Base Accuracy:  1.6532775528224812


Random Accuracy:  1.8348417320461654
Improvement of 0.11%.


RF_Randomized_Search_CV




# 효준이 포함 성능

In [18]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)

In [19]:
regressor.fit(X_train_new, y_train)  

RandomForestRegressor(random_state=0)

In [20]:
y_pred_new = regressor.predict(X_test_new)  # test the output by changing values

In [21]:
RMSE_new = mean_squared_error(y_test, y_pred_new)**0.5

In [22]:
RMSE_new

1.422598783916282